In [8]:
from transformers import BertTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments, BertForMaskedLM
from datasets import load_dataset
import datasets
import torch
import re
from nltk.tokenize import sent_tokenize
import nltk
import accelerate

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ryanm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
dataset = load_dataset("ErikCikalleshi/new_york_times_news_1987_1995", split='test[:1%]')

In [3]:
unique_dates = list(set(sorted(dataset['date'])))
custom_date_tokens = [f"<year_{d}>" for d in unique_dates]
model_name = "bert-base-uncased"
custom_token = custom_date_tokens
tokenizer = BertTokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({'extra_special_tokens' : custom_date_tokens})


9

In [4]:
sentenceData = {'text': []}
for entry in dataset:
    text = entry['content'] # type: ignore
    date = entry['date'] # type: ignore
    for sentence in sent_tokenize(text):
        sentence = f'<year_{date}> '+ sentence
        sentence = sentence[:min(512, len(sentence))]
        sentenceData['text'].append(sentence)
tokenized_dataset = datasets.Dataset.from_dict(sentenceData)


In [5]:
def tokenize_data(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result



In [11]:
# Data collator for Masked Language Modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

tokenized_dataset = tokenized_dataset.map(tokenize_data, batch_size=64, batched=True )

# Load pre-trained model
model = BertForMaskedLM.from_pretrained("bert-base-uncased")
model.resize_token_embeddings(len(tokenizer))

Loading weights: 100%|██████████| 202/202 [00:00<00:00, 1105.44it/s, Materializing param=cls.predictions.transform.dense.weight]                 
BertForMaskedLM LOAD REPORT from: bert-base-uncased
Key                         | Status     |  | 
----------------------------+------------+--+-
bert.pooler.dense.weight    | UNEXPECTED |  | 
cls.seq_relationship.weight | UNEXPECTED |  | 
bert.pooler.dense.bias      | UNEXPECTED |  | 
cls.seq_relationship.bias   | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(30531, 768, padding_idx=0)

In [12]:
training_args = TrainingArguments(
    output_dir="./domain_pretrained_model",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

# Pretrain the model
trainer.train()

c:\Users\ryanm\Comp Sci Work\Semantics-Research\.venv\Lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


Step,Training Loss


KeyboardInterrupt: 